In [ ]:
ScAT tutorial

## 00 QC Control

In [ ]:
source /YOUR_CONDA_PATH/anaconda3/bin/activate
conda activate py3
rscript=/YOUR_CONDA_PATH/anaconda3/envs/R411/bin/Rscript
scat=/PATH_to_ScAT/ScAT/core/ScAT

gem_file=/YOUR_PATH/FP200000571BR_A2_1_bin50.gem
path=/YOUR_OUTPUT_PATH/
prefix=mouse_brain_bin50

###################00 QC Control#############################
# step 1 Preview the data and obtain raw_seurat_object.rds
python3 $scat QcControl \
    -i $gem_file \
    -o ${path}/00.QC/step1 \
    --binsize 50 \
    --prefix $prefix \
    --mtPattern '^Mt' \
    --rbPattern '^Rp[sl]' \
    --hbPattern '^Hb[^(p)]' \
    --platPattern 'Pecam1|Pf4' \
    --ptsize 50


# step 2 Perform filteraion and normalization 
# Note! You need to manually set the filter parameters based on the QC metrics generated in the previous step.
# ~2.2G
python3 $scat QcControl \
    -i ${path}/00.QC/step1/${prefix}_raw_seuratObject.rds \
    -o ${path}/00.QC/step2 \
    --prefix $prefix \
    --mFilter \
    --mtPattern '^Mt' \
    --rbPattern '^Rp[sl]' \
    --hbPattern '^Hb[^(p)]' \
    --platPattern 'Pecam1|Pf4' \
    --minCell 3 \
    --ptsize 50

###################00 QC Control#############################

## 01 Clustering

In [ ]:
####################01 Clustering#############################
# input: flt_norm_seurat_object.rds, assays: SCT
python3 $scat Clustering \
    -i ${path}/00.QC/step2/${prefix}_filt_norm_seuratObject.rds \
    -o ${path}/01.Clustering \
    --prefix $prefix 

####################01 Clustering#############################

## 02 Trajectory

In [ ]:
####################02 Trajectory#############################
python3 $scat Trajectory \
    -i ${path}/01.Clustering/${prefix}_filt_norm_cluster_seuratObject.rds \
    -o ${path}/02.trajectory \
    --prefix $prefix \
    --mDEG

####################02 Trajectory#############################

## 03 Annotation

In [ ]:
####################03 Annotation#############################
python3 $scat Annotation \
    -i ${prefix}/00.QC/sc1_out/flt_norm/brain_filt_norm_matrix.txt.gz \
    -o ${path}/03.annotation/sc1_out \
    --prefix $prefix \
    --ref_name  'HumanPrimaryCellAtlasData' \
    --cluster ${prefix}/01.Clustering/sc1_out/brain_clusterInfo.txt 

####################03 Annotation#############################

## 04 Enrichment

In [ ]:
####################04 Enrichment#############################
python3 $scat Enrichment\
    -i ${path}/e15_matrix.txt.gz\
    -m ${path}/cell_types.txt\
    -s 'Human'\
    -o ${path}/03.Enrichment\

####################04 Enrichment#############################

## 05 CellCellInteraction

In [ ]:
####################05 CellCellInteraction#############################
python3 $scat CellCellInteraction \
    -i ${path}/00.QC/step1/${prefix}_filt_norm_matrix.txt.gz \
    -o ${path}/04.CellCellInteraction/sc1_out \
    -m ${path}/01.Clustering/${prefix}_clusterInfo_CCI.txt \
    -s 'Human' \
    --prefix $prefix

####################05 CellCellInteraction#############################

## 06 TFregulon

In [ ]:
######################06  TFregulon###############################
python3 $scat TFregulon \
    --species human\
    --GeneMatrix ${path}/00.QC/step2${prefix}_filt_matrix.txt \
    --annot ${path}/01.Clustering/${prefix}_clusterInfo_TF.txt \
    --TFDataset /YOUR_PATH/ScAT/docs/cisTarget_databases \
    --cpu 10 \
    --outdir ${path}/03.tf

######################06  TFregulon###############################


python3 $scat TFregulon \
    --species human\
    --TFreglist ${path}/03.tf/specify.TFregulon.example.txt \
    --outdir ${path}/03.tf \
    
    
python3 $scat TFregulon \
    --TFName IRF1 \
    --species human\
    --SCENICloom ${path}/03.tf/04.SCENIC.Regulon.AUC.loom \
    --h5ad ${path}/03.tf/05.Adata.Expression.Regulon.AUC.h5ad \
    --NetFile ${path}/03.tf/02.Network.Ad1.tsv \
    --outdir ${path}/03.tf \    
   

## 07 DeconvSpot

In [ ]:
######################08 DeconvSpot###############################
python3 $scat DeconvSpot \
    --Scdata $sc_rds\
    --Stdata $st_rds\
    --Scdata $sc_rds\   
    --ClusterFile ${path}/08.DeconvSpot/specify.cluster.example.txt\   
    --piescale 1.2\
    --outdir ${path}/08.DeconvSpot

######################08 DeconvSpot###############################

## 08 SpatialCluster

In [ ]:
######################09 SpatialCluster###############################
python3 $scat SpatialCluster\
    --Stdata $st_rds\
    --clusterNumber 8\
    --interations 10000\   
    --outdir ${path}/09.SpatialCluster

######################09 SpatialCluster###############################

## 09 SpatialPatternGene

In [ ]:
######################10 SpatialPatternGene###############################
python3 $scat SpatialPatternGene\
    --input $st_rds\
    --outdir ${path}/10.SpatialPatternGene

python3 $scat SpatialPatternGene\
    --input $st_rds\
    --genefile ${path}/test.gene.txt \
    --outdir ${path}/10.SpatialPatternGene

######################10 SpatialPatternGene###############################